# Quotation Extraction: Version 2
## 0 Imports

In [1]:
import numpy
import csv
# spaCy
import spacy
# spaCy Visualizer
from spacy import displacy

## 1 Text processing
### 1.1 Loading and cleaning the text file
Loads the article, removes all new line characters and replaces all variants of quotes by a unique one. Puts the text through the NLP pipeline.

I have also noticed that the model doesn't deal with ";" symbols to seperate sentences, which it sometimes treats as the end of the sentence and sometimes not. I replace them with commas (",").

In [2]:
quotes = ["«", "»", "“", "”", "„", "‹", "›", "‟", "〝", "〞"]

# Replace all formats of quotation marks by the quotation mark <">
def normalize_quotes(text):
    for q in quotes:
        text = text.replace(q, '"')
    return text


with open('../data/article.txt', 'r') as file:
    text = normalize_quotes(file.read().replace('\n', ' '))
    text = text.replace(";", ",")

nlp = spacy.load("fr_core_news_md")
doc = nlp(text)

### 1.2 Splitting text into sentences
Splits the text into a list of sentences

In [3]:
sentences = list(doc.sents)

## 2 Loading Model Parameters
### 2.1 Loading cue verbs

In [4]:
with open('../data/cue_verbs.csv', 'r') as f:
    reader = csv.reader(f)
    cue_verbs = set(list(reader)[0])

print(cue_verbs)

{'constater', 'concevoir', 'remarquer', 'déclarer', 'proposer', 'noter', 'prouver', 'abonder', 'calculer', "mettre l'accent sur", 'analyser', 'dénoncer', 'définir', 'considérer', 'identifier', 'prôner', 'suggérer', 'préciser', 'déplorer', 'élaborer', 'formuler', 'estimer', 'recenser', 'répondre', 'résumer', 'rapporter', 'exprimer', 'établir', 'découvrir', 'soutenir', 'adopter', "s'interroger", 'commenter', 'prétendre', 'présenter', 'raconter', 'ajouter', 'reconnaître', 'relever', 'souligner', 'observer', 'montrer', 'affirmer', 'annoncer', 'démontrer', 'écrire', 'expliquer', 'proclamer', 'citer', 'corroborer', 'dire', 'donner', 'décrire', 'mettre en évidence', 'publier', 'crier', 'étudier', 'insister'}


### 2.2 Loading Quotations Structures
The quotation structures are stored in a CSV file, respecting the following format. All Part-Of-Speech elements are abbreviated as follows.
* RS: Reported Speech
* CV: Cue Verb
* QT: Quotee
* text: distinct words
One structure per line, with each element seperated by a comma.

In [5]:
with open('../data/quote_structures.csv', 'r') as f:
    reader = csv.reader(f)
    structures = list(reader)

print(structures)

[['parataxis', 'obj', 'ancestor'], ['parataxis', 'nsubj', 'ancestor'], ['ROOT', 'obj', 'advcl']]


## 3 Finding quote containing sentences
### 3.1 Finding sentences containing cue verbs

In [6]:
# Returns true iff the sentence contains quotation marks.
def contains_cue(sentence):
    for token in sentence:
        if token.lemma_ in cue_verbs:
            return True


cues = []
        
for s in sentences:
    if contains_cue(s):
        cues.append(s)
        print(s, "\n")

Mais "les nuits de pleine lune, les plus claires d’entre elles resplendissent comme des soleils", observe Alexandre Roulin, de l’Université de Lausanne. 

La réponse a été publiée dans la revue Nature Ecology & Evolution. 

"Nous avons installé 400 nichoirs contre des granges et suivi le devenir de plus de 1000 couvées: œufs pondus, poids des nichées, proies rapportées chaque nuit par le mâle – qui est le seul à chasser…" 

A l’aide de caméras infrarouges, ils ont d’abord montré que les mâles rapportaient au nid, en moyenne, 4,78 proies par nuit. 

"Contre toute attente, leurs performances n’étaient pas affectées par les nuits de pleine lune", souligne Luis San-Jose, premier auteur de l’étude. 

Mais quand l’effort de chasse était soutenu, tout changeait. 

Dans le même temps, nous avons fait "voler" au-dessus de leurs têtes, à l’aide de tyroliennes, des chouettes empaillées de différentes couleurs", raconte Alexandre Roulin. 

Et les chouettes blanches ont plus de facilité pour les at

### 3.2 Determining which sentences contain quotes
Using the structures and cue verbs that were manually found, for each sentence containing a cue verb we output if a quote was found, or if no quote is in the sentence.

In [8]:
def extract_cue(sentence):
    for token in sentence:
        if token.lemma_ in cue_verbs:
            return token

def extract_quotee(token):
    quotee = ""
    for t in token.subtree:
        quotee += t.text + t.whitespace_
    return quotee

def quote_text(token, cv_dep):
    quote = ""
    for t in token.lefts:
        if t.dep_ != cv_dep:
            quote += quote_text(t, cv_dep)
    quote += token.text + token.whitespace_
    for t in token.rights:
        if t.dep_ != cv_dep:
            quote += quote_text(t, cv_dep)
    return quote

def is_quote(sentence):
    if contains_cue(sentence):
        cv = extract_cue(sentence)
        children = list(map(lambda t:t.dep_, list(cv.children)))
        for struct in structures:
            cv_dep = struct[0]
            qt_pos = struct[1]
            q_pos = struct[2]
            if cv.dep_ == cv_dep and qt_pos in children:
                for child in cv.children:
                    if child.dep_ == qt_pos:
                        qt_token = child
                quotee = extract_quotee(qt_token)
                if q_pos == "ancestor":
                    quote = quote_text(next(cv.ancestors), cv_dep)
                else:
                    for child in cv.children:
                        if child.dep_ == q_pos:
                            q_token = child
                    quote = quote_text(q_token, cv_dep)
                return quotee + " said " + quote
    return "No quote in this sentence"

for s in cues:
    print("Sentence:\n")
    print(s.text, "\n")
    print("The result was:\n")
    print(is_quote(s), "\n\n\n")

Sentence:

Mais "les nuits de pleine lune, les plus claires d’entre elles resplendissent comme des soleils", observe Alexandre Roulin, de l’Université de Lausanne. 

The result was:

Alexandre Roulin said Mais "les nuits de pleine lune, les plus claires d’entre elles resplendissent comme des soleils", .  



Sentence:

La réponse a été publiée dans la revue Nature Ecology & Evolution. 

The result was:

No quote in this sentence 



Sentence:

"Nous avons installé 400 nichoirs contre des granges et suivi le devenir de plus de 1000 couvées: œufs pondus, poids des nichées, proies rapportées chaque nuit par le mâle – qui est le seul à chasser…" 

The result was:

No quote in this sentence 



Sentence:

A l’aide de caméras infrarouges, ils ont d’abord montré que les mâles rapportaient au nid, en moyenne, 4,78 proies par nuit. 

The result was:

No quote in this sentence 



Sentence:

"Contre toute attente, leurs performances n’étaient pas affectées par les nuits de pleine lune", souligne